# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 12h Sleeping Hypothalamus Expression values and Group 2 be 12h SD Hypothalamus Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_HypoT_12h_'   # define a prefix to add to column names (making indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE6514.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 137)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 137)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'adjp-HypoT_12hS_vs_HypoT_12hSD', 'GSM149636_HypoT_12hSD.CEL',
       'GSM149637_HypoT_12hSD.CEL', 'GSM149648_HypoT_12hSD.CEL',
       'GSM149649_HypoT_12hSD.CEL', 'GSM149650_HypoT_12hSD.CEL',
       'avg-HypoT_12hSD', 'ANOVA-rawp', 'ANOVA-adjp', 'largest fold'],
      dtype='object', length=137)

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='HypoT_12hS.CEL'
sd_filt ='HypoT_12hSD.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM149631_HypoT_12hS.CEL,GSM149632_HypoT_12hS.CEL,GSM149633_HypoT_12hS.CEL,GSM149634_HypoT_12hS.CEL,GSM149635_HypoT_12hS.CEL
Probesets,,,,,
1427138_at,6.62696,6.61806,6.57883,6.54142,6.33934
1425600_a_at,7.40271,7.35949,7.11284,7.06383,7.17386
1457168_at,5.95149,6.00765,5.76789,5.68816,5.74562
1450135_at,5.90148,6.05079,5.89562,5.94316,5.84329
1424014_at,7.16697,7.77256,7.31122,7.68999,7.86428


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM149636_HypoT_12hSD.CEL,GSM149637_HypoT_12hSD.CEL,GSM149648_HypoT_12hSD.CEL,GSM149649_HypoT_12hSD.CEL,GSM149650_HypoT_12hSD.CEL
Probesets,,,,,
1427138_at,6.79241,6.73521,6.41185,6.46732,6.85346
1425600_a_at,7.34673,7.62411,7.14734,7.41166,7.83126
1457168_at,5.68893,6.01570,5.89278,5.86310,5.81704
1450135_at,5.75811,5.90879,5.57141,6.01366,5.84724
1424014_at,8.33823,7.44405,7.70478,7.73172,7.71696


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at      0.111128
1425600_a_at    0.249674
1457168_at      0.023348
1450135_at     -0.107026
1424014_at      0.226144
Name: 430AV2_HypoT_12h_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at      0.678776
1425600_a_at    1.164504
1457168_at      0.180585
1450135_at     -0.820896
1424014_at      0.711927
Name: 430AV2_HypoT_12h_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at      0.622212
1425600_a_at    1.067462
1457168_at      0.165537
1450135_at     -0.752488
1424014_at      0.652600
Name: 430AV2_HypoT_12h_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.423037
1425600_a_at    0.467803
1457168_at      0.401631
1450135_at      0.433693
1424014_at      0.425342
Name: 430AV2_HypoT_12h_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_HypoT_12h_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_HypoT_12h_Enrich,430AV2_HypoT_12h_poolStDev,430AV2_HypoT_12h_Cohens_d,430AV2_HypoT_12h_J,430AV2_HypoT_12h_Hedges_g,430AV2_HypoT_12h_Var_d,430AV2_HypoT_12h_Var_g,430AV2_HypoT_12h_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1457495_at,2900052N01Rik,RIKEN cDNA 2900052N01 gene,NaN,73040,Mm.265384,NaN,NaN,NaN,NaN,NaN,...,4.199865e-09,0.632368,0.389340,0.070250,5.542181,0.916667,5.080333,1.935788,1.626600,1.275382
1436387_at,C330006P03Rik,homer homolog 1 (Drosophila) [Source:MGI Symbo...,ENSMUSG00000007617,320588,NaN,NaN,NaN,NaN,NaN,"mmu-let-7a(RNAhybrid|miRanda), mmu-let-7b(RNAh...",...,5.979233e-29,5.778282,2.855228,0.555049,5.144104,0.916667,4.715429,1.723091,1.447875,1.203277
1428471_at,Sorbs1,sorbin and SH3 domain containing 1,ENSMUSG00000025006,20411,Mm.210815,focal adhesion formation // insulin receptor s...,protein binding // insulin receptor binding //...,cell-cell adherens junction // stress fiber //...,GenMAPP-Integrin-mediated_cell_adhesion_KEGG,"mmu-miR-101a(miRanda), mmu-miR-125a-3p(miRanda...",...,1.284490e-09,1.100552,0.389286,0.087325,4.457897,0.916667,4.086406,1.393642,1.171047,1.082149
1437913_at,Bcl2a1a /// Bcl2a1b /// Bcl2a1d,B cell leukemia/lymphoma 2 related protein A1a,ENSMUSG00000074145|ENSMUSG00000074147|ENSMUSG0...,12047|12044|12045,NaN,regulation of apoptosis // apoptosis // negati...,magnesium ion binding // protein binding // li...,cytoplasm,GenMAPP-Apoptosis_KEGG,"mmu-miR-291a-3p(miRanda), mmu-miR-294(miRanda)...",...,1.496092e-01,0.190766,0.176100,0.040613,4.336068,0.916667,3.974729,1.340074,1.126035,1.061148
1439906_at,NaN,ets variant gene 1 [Source:MGI Symbol;Acc:MGI:...,ENSMUSG00000004151,NaN,NaN,NaN,NaN,NaN,NaN,"mmu-miR-101(TargetScan), mmu-miR-101ab(TargetS...",...,3.060524e-09,1.221362,0.314356,0.075246,4.177699,0.916667,3.829558,1.272659,1.069387,1.034112
1436045_at,Tsga10,testis specific 10 [Source:MGI Symbol;Acc:MGI:...,ENSMUSG00000060771,211484,NaN,cell projection assembly,structural molecule activity,neuron projection // cytoplasm // nucleus // n...,XPodNet - protein-protein interactions in the ...,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,2.324115e-10,0.818932,0.405362,0.102027,3.973101,0.916667,3.642009,1.189277,0.999323,0.999661
1416749_at,Htra1,HtrA serine peptidase 1,ENSMUSG00000006205,56213,NaN,negative regulation of transforming growth fac...,insulin-like growth factor binding // serine-t...,extracellular region,XPodNet - protein-protein interactions in the ...,"mmu-let-7i(RNAhybrid|miRanda), mmu-miR-106b(RN...",...,2.740669e-06,0.629486,0.419698,0.106637,3.935746,0.916667,3.607767,1.174505,0.986910,0.993434
1418526_at,Fusip1,FUS interacting protein (serine-arginine rich) 1,ENSMUSG00000028676,14105,NaN,"negative regulation of nuclear mRNA splicing, ...",nucleic acid binding // RNA splicing factor ac...,nucleoplasm // nucleus // nuclear speck,mRNA processing:WP310(WikiPathways),"mmu-let-7b(miRanda), mmu-miR-101(pictar), mmu-...",...,9.867948e-07,0.787090,0.322174,0.081963,3.930739,0.916667,3.603178,1.172536,0.985256,0.992600
1441360_at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.603164e-09,1.008286,0.322930,0.083898,3.849075,0.916667,3.528319,1.140769,0.958563,0.979062


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'ANOVA-adjp', 'largest fold', '430AV2_HypoT_12h_Enrich',
       '430AV2_HypoT_12h_poolStDev', '430AV2_HypoT_12h_Cohens_d',
       '430AV2_HypoT_12h_J', '430AV2_HypoT_12h_Hedges_g',
       '430AV2_HypoT_12h_Var_d', '430AV2_HypoT_12h_Var_g',
       '430AV2_HypoT_12h_SEg'],
      dtype='object', length=145)

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM149516_CerCx_3hS.CEL,GSM149517_CerCx_3hS.CEL,GSM149518_CerCx_3hS.CEL,GSM149519_CerCx_3hS.CEL,GSM149520_CerCx_3hS.CEL,avg-CerCx_3hS,log_fold-CerCx_3hS_vs_CerCx_3hSD,fold-CerCx_3hS_vs_CerCx_3hSD,rawp-CerCx_3hS_vs_CerCx_3hSD,adjp-CerCx_3hS_vs_CerCx_3hSD,...,ANOVA-adjp,largest fold,430AV2_HypoT_12h_Enrich,430AV2_HypoT_12h_poolStDev,430AV2_HypoT_12h_Cohens_d,430AV2_HypoT_12h_J,430AV2_HypoT_12h_Hedges_g,430AV2_HypoT_12h_Var_d,430AV2_HypoT_12h_Var_g,430AV2_HypoT_12h_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_HypoT_12h_Enrich',u'430AV2_HypoT_12h_Hedges_g', u'430AV2_HypoT_12h_Var_g', u'430AV2_HypoT_12h_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_HypoT_12h_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_HypoT_12h_Enrich,430AV2_HypoT_12h_Hedges_g,430AV2_HypoT_12h_Var_g,430AV2_HypoT_12h_SEg
MGI symbol,,,,
0610005C13Rik,-0.063320,-0.676177,0.358972,0.599143
0610008F07Rik,-0.073018,-0.479205,0.347593,0.589570
0610009B22Rik,-0.038908,-0.218760,0.338504,0.581811
0610009D07Rik,0.063857,0.204384,0.339106,0.582323
0610009O20Rik,-0.041124,-0.398998,0.344071,0.586576
0610010K14Rik,0.008926,0.037443,0.336181,0.579811
0610012G03Rik,0.196344,0.712955,0.362951,0.602375
0610031J06Rik,-0.041340,-0.152844,0.337279,0.580757
0610037L13Rik,-0.033930,-0.337580,0.341809,0.584644
